In [9]:
from pathlib import Path
from dotenv import load_dotenv

import pandas as pd
import glob
import os

load_dotenv("../.env")

True

In [4]:
DATA_CSV = Path("../data/csv/")

# Level 2: Processing CSV

**Note**  
Low level dataframes processing (like converting datetimes, filtering, dropping duplicates, sorting) is done in the previous notebook. Here we assume that the data is already cleaned and ready for further processing.

In [12]:
csv_paths = glob.glob(os.path.join(DATA_CSV, "*.csv"))
for path in csv_paths:
    df = pd.read_csv(path)
    print(df)

                       time  CEU Temperature (1)  P&IDH Temperature (2)  \
0       2023-09-01 00:00:03                -20.0                  -19.0   
1       2023-09-01 00:01:03                -20.0                  -19.0   
2       2023-09-01 00:02:03                -20.0                  -19.0   
3       2023-09-01 00:03:03                -20.0                  -18.0   
4       2023-09-01 00:04:03                -20.0                  -19.0   
...                     ...                  ...                    ...   
356649  2024-05-07 15:33:21                -13.0                  -11.0   
356650  2024-05-07 15:34:21                -13.0                  -11.0   
356651  2024-05-07 15:35:21                -13.0                  -11.0   
356652  2024-05-07 15:36:21                -13.0                  -11.0   
356653  2024-05-07 15:37:21                -13.0                  -11.0   

        EDH Temperature (3)  DDH Temperature (4)  PCU Temperature (5)  
0                     -18.0